# Nestle - Zadanie Rekturacyjne
## Business Case - Data Science Hub
### Autor: Szymon Baczyński

## 1. Wczytanie danych i przegląd
### Pliki treningowe: X_train.csv & Y_train.csv
### Pliki testowe: X_test.csv & Y_test.csv

In [ ]:
# import datatable as dt      # For bigger files - data.table - faster (or use dplyr)

import pandas as pd
import random
import itertools
import numpy as np
import xgboost as xgb
from datetime import datetime
import requests
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
import IPython.display as Disp
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

sns.set_theme()

In [ ]:
# Wczytanie danych i szybki przeglad
X_train = pd.read_csv("X_train.csv", delimiter= ";", decimal=",")
X_train.info()
X_train.head()

In [ ]:
# Rozmiar i nazwy kolumn
X_train.shape
X_train.columns

In [ ]:
# ID produktow w danych treningowych
X_train.key.unique()

# Ilosc unikalnych produktow
X_train.key.nunique()

In [ ]:
# Sprawdzenie zakresu dat
X_train.date.unique()

In [ ]:
# Sprawdzenie co sie dzieje w danych - nowe produkty (w czasie) + zmienna ilosc danych dla poszczegolnych produktow
X_train.groupby('key').count()

In [ ]:
# Ile jest wartosci w poszczegolnych kolumnach -> nie ma problemu w 'key' i 'date'
X_train.isnull().sum()
# i ilosc wartosci nieliczbowych dla poszczegolnych produktow
# pd.concat([X_train.loc[:,'key'], X_train.isnull().sum(axis=1)], axis=1).groupby('key').sum()

In [ ]:
# Sprawdzmy przy okazji dane "target" -> "Y"
Y_train = pd.read_csv("Y_train.csv", delimiter= ";", decimal=",")
Y_train.info()
Y_train.head()

In [ ]:
# Zobaczmy ile jest wartosci null czy NaN
Y_train.isnull().sum()
Y_train[Y_train.isnull().any(axis=1)]
Y_train[Y_train.isnull().any(axis=1)].groupby('key').count()

In [ ]:
# Dane "Y" - produkty tez maja rozna ilosc dat
Y_train.groupby('key').count()

In [ ]:
# Sprawdzmy czy ilosc dat dla X i Y w zbiorze treningowym sie zgadzaja
all(X_train.groupby('key').count().date == Y_train.groupby('key').count().date)

In [ ]:
# Zduplikowane wiersze "key" i "date" ?
X_train.duplicated(subset=['key','date']).any()
Y_train.duplicated(subset=['key','date']).any()

## 2. Data Wrangling - zróbmy z tego dane, które da się czytać

In [ ]:
url = 'https://miro.medium.com/max/400/0*TXDoF8j-D3LuGbHP.jpg'
Disp.Image(requests.get(url).content)

In [ ]:
# Naprawmy kolumne z datami
X_train['date'] = pd.to_datetime(X_train['date'], format='%d%b%Y')
X_train.info()
X_train.head()

In [ ]:
# Kopia X_train aby uzupelnic wartosci NaN
X_train_copy = X_train.copy(deep=True)

In [ ]:
# Uzupelnianie wartosci NaN

col_names = X_train.iloc[:,2:].columns
X_train_copy[col_names] = X_train_copy.groupby('key')[col_names].transform(lambda x: x.fillna(x.rolling(3,min_periods=1).mean()))

X_train_copy.fillna(0.0, inplace=True)

### Znalezc jak sie robi fillna z 0 na poczatku a pozniej innymi wartosciami


In [ ]:
# Poprawiona data w "Y"
Y_train['date'] = pd.to_datetime(Y_train['date'], format='%d%b%Y')

In [ ]:
Y_train_copy = Y_train.copy(deep=True)

In [ ]:
# Uzupelnienie wartosci "Y"
Y_train_copy['y'] = Y_train_copy.groupby('key')['y'].transform(lambda x: x.fillna(x.rolling(3,min_periods=1).mean()))

Y_train_copy.fillna(0.0, inplace=True)

In [ ]:
Y_train_copy
Y_train_copy.isnull().sum()

Notatki:
- Widać ubytki w danych, często dość kluczowe - można by zastosować średnią kroczącą, w celu uzupełnienia 
- Niestety takie uzupełnianie może nie być prawdziwe - warto zapytać klienta czy nie ma tych danych
- W przypadku gdy brak danych, można uzupełnić je wartościami średnimi z małym okienkiem (pozostałe wartości uzupełnione 0.0)
- W dalszej części zostanie zrobiona eksploracja obu zbiorów danych by sprawdzić jak uzupełnione wartości wpływają na zbiór

## 3. Exploratory Data Analysis - sprawdźmy dane pod kątem statystyki

In [ ]:
X_train_copy.info()
X_train_copy.head()

# ID produktow w danych treningowych
X_train_copy.key.unique()

# Ilosc unikalnych produktow
X_train_copy.key.nunique()

In [ ]:
# Wglad w dane "Y"
Y_train.info()
Y_train.head()

In [ ]:
# Plot for random product (key)
plt.plot(Y_train[Y_train['key'] == 72159].date, Y_train[Y_train['key'] == 72159].y)
plt.plot(Y_train[Y_train['key'] == 9902].date, Y_train[Y_train['key'] == 9902].y)
plt.plot(Y_train[Y_train['key'] == 99444].date, Y_train[Y_train['key'] == 99444].y)
plt.legend(['72159','9902','99444'])
plt.show();

In [ ]:
# Czy w "Y" sa outliers
sns.boxplot(x=Y_train['y'])

In [ ]:
plot_acf(Y_train_copy[Y_train_copy['key'] == 72159].y);
plot_pacf(Y_train_copy[Y_train_copy['key'] == 72159].y);

In [ ]:
plot_acf(Y_train_copy[Y_train_copy['key'] == 9902].y);
plot_pacf(Y_train_copy[Y_train_copy['key'] == 9902].y);

In [ ]:
plot_acf(Y_train_copy[Y_train_copy['key'] == 99444].y);
plot_pacf(Y_train_copy[Y_train_copy['key'] == 99444].y);

In [ ]:
# Roznica miedzy zbiorem z NaN i z uzupelnionymi wartosciami
X_train.loc[X_train['key'] == 683].isnull().sum().sum()
X_train_copy.loc[X_train_copy['key'] == 683].isnull().sum().sum()
X_train.isnull().sum().sum()
X_train_copy.isnull().sum().sum()

In [ ]:
X_train.iloc[:,2:].describe()
X_train_copy.iloc[:,2:].describe()

In [ ]:
# Plot some variables 'x' for random keys to show missing data
rand_keys = random.choices(X_train['key'].unique(), k=5)
rand_keys

for keys in rand_keys:
    plt.plot(X_train.loc[X_train['key'] == keys].date, X_train.loc[X_train['key'] == keys].iloc[:,3:10]);
    plt.legend(X_train.iloc[:,3:10].columns);
    plt.show();
    
    plt.plot(X_train_copy.loc[X_train_copy['key'] == keys].date, X_train_copy.loc[X_train_copy['key'] == keys].iloc[:,3:10]);
    plt.legend(X_train_copy.iloc[:,3:10].columns);
    plt.show();

In [ ]:
X_Y_train_with_NaN = X_train.copy(deep=True)
X_Y_train_with_NaN.insert(2,'y',Y_train.y)
X_Y_train_with_NaN

In [ ]:
# X_Y_train_without_NaN = pd.concat([X_train_copy, Y_train_copy.y], axis=1)
X_Y_train_without_NaN = X_train_copy.copy(deep=True)
X_Y_train_without_NaN.insert(2,'y',Y_train_copy.y)
X_Y_train_without_NaN

In [ ]:
# Korelacja z NaN

Train_corr_NaN = X_Y_train_with_NaN.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(Train_corr_NaN, cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(X_Y_train_with_NaN.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(X_Y_train_with_NaN.columns)
ax.set_yticklabels(X_Y_train_with_NaN.columns)
plt.show();

In [ ]:
# Sprawdzenie korelacji 'y' z 'x' dla danych z NaN
corr_NaN = X_train.corrwith(Y_train['y'])
corr_NaN
plt.plot(corr_NaN);

In [ ]:
# Korelacja dla danych bez NaN

Train_corr = X_Y_train_without_NaN.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(Train_corr, cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(X_Y_train_without_NaN.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(X_Y_train_without_NaN.columns)
ax.set_yticklabels(X_Y_train_without_NaN.columns)
plt.show();

In [ ]:
# Korelacja 'y' z 'x' dla danych bez NaN - tracimy korelacje danych przez uzupelnienie wartosci NaN
corr_without_NaN = X_train_copy.corrwith(Y_train_copy['y'])
plt.plot(corr_without_NaN);
corr_without_NaN[corr_without_NaN > 0.15]

In [ ]:
corr_tmp_max = list()

for k in range(0,13):
    tmp = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
    corr_tmp = tmp.iloc[:,3:].shift(k, fill_value=0.0).corrwith(tmp['y'])
    plt.plot(corr_tmp)
    corr_tmp_max.append((corr_tmp[corr_tmp > 0.5]))

plt.legend(range(0,13))
plt.show();


In [ ]:
pd.DataFrame(corr_tmp_max)

In [ ]:
tmp = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp = tmp.iloc[:,3:].corrwith(tmp['y'])
plt.plot(corr_tmp);
corr_tmp[corr_tmp > 0.5]

In [ ]:
# Korelacja wzgledem przesunietych 'X'

tmp_1 = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp_1 = tmp_1.iloc[:,3:].shift(1, fill_value=0.0).corrwith(tmp_1['y'])
plt.plot(corr_tmp_1);
corr_tmp_1[corr_tmp_1 > 0.5]


In [ ]:
tmp_2 = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp_2 = tmp_2.iloc[:,3:].shift(2, fill_value=0.0).corrwith(tmp_2['y'])
plt.plot(corr_tmp_2);
corr_tmp_2[corr_tmp_2 > 0.5]

In [ ]:
tmp_3 = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp_3 = tmp_3.iloc[:,3:].shift(3, fill_value=0.0).corrwith(tmp_3['y'])
plt.plot(corr_tmp_3);
corr_tmp_3[corr_tmp_3 > 0.5]

In [ ]:
tmp_4 = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp_4 = tmp_4.iloc[:,3:].shift(4, fill_value=0.0).corrwith(tmp_4['y'])
plt.plot(corr_tmp_4);
corr_tmp_4[corr_tmp_4 > 0.5]

In [ ]:
tmp_5 = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp_5 = tmp_5.iloc[:,3:].shift(5, fill_value=0.0).corrwith(tmp_5['y'])
plt.plot(corr_tmp_5);
corr_tmp_5[corr_tmp_5 > 0.5]

In [ ]:
tmp_6 = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp_6 = tmp_6.iloc[:,3:].shift(6, fill_value=0.0).corrwith(tmp_6['y'])
plt.plot(corr_tmp_6);
corr_tmp_6[corr_tmp_6 > 0.5]

In [ ]:
tmp_7 = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp_7 = tmp_7.iloc[:,3:].shift(7, fill_value=0.0).corrwith(tmp_7['y'])
plt.plot(corr_tmp_7);
corr_tmp_7[corr_tmp_7 > 0.5]

In [ ]:
tmp_8 = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == 9902]
corr_tmp_8 = tmp_8.iloc[:,3:].shift(8, fill_value=0.0).corrwith(tmp_8['y'])
plt.plot(corr_tmp_8);
corr_tmp_8[corr_tmp_8 > 0.5]

In [ ]:
# Find the best correlation 

shift_start = 1
shift_lookin = 9

key_ID = list(np.repeat(X_train_copy['key'].unique(), shift_lookin-shift_start))
key_shift = list(itertools.chain.from_iterable(list(itertools.repeat(list(range(shift_start,shift_lookin)), X_train_copy.key.nunique()))))

In [ ]:
# Check which 'x' is important (corr > 0.5) for every 'key'

corr_list = list()

for key in X_train_copy['key'].unique():
    for m_shift in range(shift_start,shift_lookin):
        tmp = X_Y_train_without_NaN[X_Y_train_without_NaN['key'] == key]
        corr_tmp = tmp.iloc[:,3:].shift(m_shift, fill_value=0.0).corrwith(tmp['y'])
        corr_list.append(list(corr_tmp))

corr_max = pd.DataFrame(corr_list, columns=X_train_copy.columns[2:])
corr_max.insert(0,'key', key_ID)
corr_max.insert(1,'shift', key_shift)

In [ ]:
corr_max

In [ ]:
corr_max.describe()


In [ ]:
# Korelacja Dodatnia
corr_max_sum = (corr_max.iloc[:,2:] > 0.5).sum()
plt.plot(corr_max_sum);

corr_max_sum[corr_max_sum > 5]


In [ ]:
# Ujemna Korelacja 

corr_min_sum = (corr_max.iloc[:,2:] < -0.5).sum()
plt.plot(corr_min_sum);

corr_min_sum[corr_min_sum > 0]

In [ ]:
# Which "shift" is the best for each 'x' - FOR ALL MODEL

columns_select = list(['shift'])
columns_select.extend(list(corr_max_sum[corr_max_sum > 5].index))

best_corr_columns = corr_max.loc[:,columns_select]
best_corr_columns.groupby('shift').max()

# plt.plot(best_corr_columns.groupby('shift').max());

df = best_corr_columns.groupby('shift').max()
fig = px.line(df, x=df.index, y=df.columns)
fig.show()

# plt.legend(list(corr_max_sum[corr_max_sum > 5].index), loc=5);

In [ ]:
# Which "shift" is the best for each 'x' - FOR MODEL PER KEY (PRODUCT)

columns_select = list(['shift'])
columns_select.extend(list(corr_max_sum[corr_max_sum > 5].index))

best_corr_columns = corr_max.loc[:,columns_select]
best_corr_columns.groupby('shift').max()

plt.plot(best_corr_columns.groupby('shift').max());
plt.legend(list(corr_max_sum[corr_max_sum > 5].index), loc=5);